In [ ]:
# Import
import streamlit as st
import cv2
from PIL import Image
import os
import psycopg2
from tensorflow.keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
import easyocr
from collections import Counter
import re

In [ ]:
# Database connection

# Function to create a PostgreSQL connection as the readonly user for querying
def get_read_only_connection():
    conn = psycopg2.connect(
        host="localhost",
        database="pokemontcg",  # Replace with your database name
        user="readonly_user",  # Shared read-only user
        password="D8G*pBDz*koJ"  # Password for the shared user
    )
    return conn

# Function to connect as the logging user for logging, retrieving credentials from environment variables
def get_logging_connection():
    conn = psycopg2.connect(
        host="localhost",
        database="pokemontcg",  # Replace with your database name
        user=os.getenv("PG_LOGGING_USER"),  # Logging user from environment variable
        password=os.getenv("PG_LOGGING_PASSWORD")  # Logging password from environment variable
    )
    return conn

# Function to log the user's activity into the restricted_logs.user_logs table
def log_user_activity(username):
    conn = get_logging_connection()  # Use the logging user connection to insert logs
    cursor = conn.cursor()

    # Insert the user's name and login time into the user_logs table
    insert_query = """
    INSERT INTO restricted_logs.user_logs (username, login_time)
    VALUES (%s, CURRENT_TIMESTAMP);
    """
    cursor.execute(insert_query, (username,))
    
    # Commit the transaction and close the connection
    conn.commit()
    cursor.close()
    conn.close()

In [ ]:
# Trained Set-Symbol Model loading
model = load_model(r"C:\Users\Jimmy\Desktop\final-project\PokemonTCG\models\model04.keras")  # Load your pre-trained model

In [ ]:
# easyReader OCR initialization
reader = easyocr.Reader(['en'])

In [ ]:
# Bounding box, ROI and OCR logic
# Function to classify a card as white or non-white
def classify_card(image_rgb):
    gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    mean_brightness = np.mean(gray)
    
    if mean_brightness > 150:
        return "white"
    else:
        return "non_white"


# Function to adjust brightness and contrast
def adjust_brightness_contrast(image, alpha=2, beta=25):
    return cv2.convertScaleAbs(image, alpha=alpha, beta=beta)


# Function to display images for debugging
def debug_show_image(image, title="Image"):
    plt.figure(figsize=(6, 6))
    plt.imshow(image, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()


# Preprocessing to identify all colors that are not black and turn them into white
def ocr_preprocessing1(image_roi):
    roi_gray = cv2.cvtColor(image_roi, cv2.COLOR_BGR2GRAY)
    _, roi_thresh_black = cv2.threshold(roi_gray, 30, 255, cv2.THRESH_BINARY)
    
    roi_contrast = adjust_brightness_contrast(roi_thresh_black, alpha=1.5, beta=30)
    roi_blur = cv2.GaussianBlur(roi_contrast, (3, 3), 0)
    roi_thresh = cv2.adaptiveThreshold(roi_blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                       cv2.THRESH_BINARY_INV, 11, 2)
    roi_sharpen = cv2.filter2D(roi_thresh, -1, np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]))
    
    debug_show_image(roi_sharpen, title="Preprocessed")
    return roi_sharpen


# OCR processing with EasyOCR
def perform_ocr_easyocr(image_roi):
    # Perform OCR using EasyOCR
    result = reader.readtext(image_roi, detail=0)
    print(f"OCR result: {result}")
    
    return result

# Function to process OCR results and select only the valid match
def process_ocr_results(ocr_results):
    # Define the patterns for the two cases
    pattern_letters_digits = r'[a-zA-Z]{2,4}\d{1,3}'  # 2-4 letters followed by 1-3 numbers
    pattern_numbers_slash = r'\d{1,3}/\d{2,3}'  # 1-3 numbers followed by / and 2-3 numbers

    for result in ocr_results:
        # Try to match both patterns
        match_letters_digits = re.search(pattern_letters_digits, result)
        match_numbers_slash = re.search(pattern_numbers_slash, result)

        # Return only the matched portion, if found
        if match_letters_digits:
            return match_letters_digits.group()  # Return the first valid match for letters-digits
        elif match_numbers_slash:
            return match_numbers_slash.group()  # Return the first valid match for numbers-slash

    return None  # Return None if no valid match is found



# Function to extract bounding boxes and run OCR on white cards
def draw_bounding_box_white(image_bgr):
    gray = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(gray, 200, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)

        # --- First Text ROI (for OCR) ---
        text_roi_height = h // 10
        text_roi_width = int(w * 0.5)
        text_roi_y_start = y + h - text_roi_height
        first_text_roi = image_bgr[text_roi_y_start:text_roi_y_start + text_roi_height, x:x + text_roi_width]

        # Debugging: show the first ROI before preprocessing
        debug_show_image(first_text_roi, title="First Text ROI Before Preprocessing")

        processed_first_text_roi = ocr_preprocessing1(first_text_roi)
        ocr_results_first = perform_ocr_easyocr(processed_first_text_roi)

        # --- Second Text ROI ---
        second_text_roi_height = int(text_roi_height * 1.2)
        second_text_roi_x_start = x + w - text_roi_width
        second_text_roi_y_start = y + h - second_text_roi_height
        second_text_roi = image_bgr[second_text_roi_y_start:second_text_roi_y_start + second_text_roi_height,
                                    second_text_roi_x_start:second_text_roi_x_start + text_roi_width]

        # Debugging: show the second ROI before preprocessing
        debug_show_image(second_text_roi, title="Second Text ROI Before Preprocessing")

        processed_second_text_roi = ocr_preprocessing1(second_text_roi)
        ocr_results_second = perform_ocr_easyocr(processed_second_text_roi)

        # --- Decision: Keep only one OCR result ---
        processed_ocr_result_first = process_ocr_results(ocr_results_first)
        processed_ocr_result_second = process_ocr_results(ocr_results_second)

        final_ocr_result = processed_ocr_result_first if processed_ocr_result_first else processed_ocr_result_second

        # --- Set Symbol ROIs ---
        roi_bottom_left = (x, y + h - text_roi_height, w // 4, text_roi_height)
        roi_bottom_right = (x + w // 2 + w // 4, y + h - 2 * text_roi_height + int(0.15 * text_roi_height), w // 4, text_roi_height)
        roi_middle = (x + w // 2 + w // 4, y + h // 2, w // 4, text_roi_height)

        # Draw bounding box and ROIs
        image_with_rois = image_bgr.copy()
        cv2.rectangle(image_with_rois, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Draw the first text ROI (blue)
        cv2.rectangle(image_with_rois, (x, text_roi_y_start), (x + text_roi_width, text_roi_y_start + text_roi_height), (255, 0, 0), 2)
        
        # Draw the second text ROI (red)
        cv2.rectangle(image_with_rois, (second_text_roi_x_start, second_text_roi_y_start), 
                      (second_text_roi_x_start + text_roi_width, second_text_roi_y_start + second_text_roi_height), (255, 0, 0), 2)

        # Draw the three set symbol ROIs (yellow)
        cv2.rectangle(image_with_rois, (roi_bottom_left[0], roi_bottom_left[1]),
                      (roi_bottom_left[0] + roi_bottom_left[2], roi_bottom_left[1] + roi_bottom_left[3]),
                      (255, 255, 0), 2)  # Yellow for set symbol ROI

        cv2.rectangle(image_with_rois, (roi_bottom_right[0], roi_bottom_right[1]),
                      (roi_bottom_right[0] + roi_bottom_right[2], roi_bottom_right[1] + roi_bottom_right[3]),
                      (255, 255, 0), 2)  # Yellow for set symbol ROI

        cv2.rectangle(image_with_rois, (roi_middle[0], roi_middle[1]),
                      (roi_middle[0] + roi_middle[2], roi_middle[1] + roi_middle[3]),
                      (255, 255, 0), 2)  # Yellow for set symbol ROI
        return image_with_rois, final_ocr_result

    else:
        print("No contours found.")
        return image_bgr, None




# Function to extract bounding boxes and run OCR on non-white cards
def draw_bounding_boxes(image_rgb, threshold_value):
    gray = cv2.cvtColor(image_rgb, cv2.COLOR_RGB2GRAY)
    _, thresholded = cv2.threshold(gray, threshold_value, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        largest_contour = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(largest_contour)

        # --- First Text ROI ---
        text_roi_height = h // 10
        text_roi_width = int(w * 0.5)
        text_roi_y_start = y + h - text_roi_height
        first_text_roi = image_rgb[text_roi_y_start:text_roi_y_start + text_roi_height, x:x + text_roi_width]

        # Debugging: show the first ROI before preprocessing
        debug_show_image(first_text_roi, title="First Text ROI Before Preprocessing (Non-White)")

        processed_first_text_roi = ocr_preprocessing1(first_text_roi)
        ocr_results_first = perform_ocr_easyocr(processed_first_text_roi)

        # --- Second Text ROI ---
        second_text_roi_height = int(text_roi_height * 1.2)
        second_text_roi_x_start = x + w - text_roi_width
        second_text_roi_y_start = y + h - second_text_roi_height
        second_text_roi = image_rgb[second_text_roi_y_start:second_text_roi_y_start + second_text_roi_height,
                                    second_text_roi_x_start:second_text_roi_x_start + text_roi_width]

        # Debugging: show the second ROI before preprocessing
        debug_show_image(second_text_roi, title="Second Text ROI Before Preprocessing (Non-White)")

        processed_second_text_roi = ocr_preprocessing1(second_text_roi)
        ocr_results_second = perform_ocr_easyocr(processed_second_text_roi)

        # --- Decision: Keep only one OCR result ---
        processed_ocr_result_first = process_ocr_results(ocr_results_first)
        processed_ocr_result_second = process_ocr_results(ocr_results_second)

        final_ocr_result = processed_ocr_result_first if processed_ocr_result_first else processed_ocr_result_second

        # --- Set Symbol ROIs ---
        roi_bottom_left = (x, y + h - text_roi_height, w // 4, text_roi_height)
        
        # Directly calculating roi_bottom_right without intermediate y calculation
        roi_bottom_right = (x + w // 2 + w // 4, y + h - 2 * text_roi_height + int(0.15 * text_roi_height), w // 4, text_roi_height)
        
        roi_middle = (x + w // 2 + w // 4, y + h // 2, w // 4, text_roi_height)

        # Draw bounding box and ROIs
        image_with_rois = image_rgb.copy()
        cv2.rectangle(image_with_rois, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # Draw the first text ROI (blue)
        cv2.rectangle(image_with_rois, (x, text_roi_y_start), (x + text_roi_width, text_roi_y_start + text_roi_height), (255, 0, 0), 2)
        
        # Draw the second text ROI (red)
        cv2.rectangle(image_with_rois, (second_text_roi_x_start, second_text_roi_y_start), 
                      (second_text_roi_x_start + text_roi_width, second_text_roi_y_start + second_text_roi_height), (255, 0, 0), 2)

        # Draw the three set symbol ROIs (yellow)
        cv2.rectangle(image_with_rois, (roi_bottom_left[0], roi_bottom_left[1]),
                      (roi_bottom_left[0] + roi_bottom_left[2], roi_bottom_left[1] + roi_bottom_left[3]),
                      (255, 255, 0), 2)  # Yellow for set symbol ROI

        cv2.rectangle(image_with_rois, (roi_bottom_right[0], roi_bottom_right[1]),
                      (roi_bottom_right[0] + roi_bottom_right[2], roi_bottom_right[1] + roi_bottom_right[3]),
                      (255, 255, 0), 2)  # Yellow for set symbol ROI

        cv2.rectangle(image_with_rois, (roi_middle[0], roi_middle[1]),
                      (roi_middle[0] + roi_middle[2], roi_middle[1] + roi_middle[3]),
                      (255, 255, 0), 2)  # Yellow for set symbol ROI
        
        return image_with_rois, final_ocr_result
    else:
        print("No contours found.")
        return image_rgb, None, None



# Main function to handle both white and non-white cards
def bounding_box_roi(image_rgb):
    card_type = classify_card(image_rgb)

    if card_type == "white":
        image_with_bounding_box, ocr_result= draw_bounding_box_white(image_rgb)
    else:
        threshold_value = 160
        image_with_bounding_box, ocr_result= draw_bounding_boxes(image_rgb, threshold_value)

    return image_with_bounding_box, ocr_result

In [ ]:
# Streamlit setup

# Set the title for the home page
st.title("Welcome to My Pokemon Card Identifier App")

# Display some text (description)
st.write("This is the home page of my app. Here you can upload an image and view it below. The image uploaded should be a single Pokemon card on a white background. The entire card should be in frame.")

# Input for name
name = st.text_input("Enter your name to log in:")

if name and not st.session_state.get('name'):
    st.session_state['name'] = name
    log_user_activity(name)  # Log the user's name and login time in the restricted schema
    st.success(f"Welcome, {name}! You are now logged in.")

# Only allow access to the rest of the app if the user has entered their name
if st.session_state.get('name'):
    # File uploader for image input
    uploaded_image = st.file_uploader("Upload an image of the card", type=["jpg", "jpeg", "png", "webp"])

    if uploaded_image is not None:
        # Open and display the uploaded image
        image = Image.open(uploaded_image)
        st.image(image, caption="Uploaded image", use_column_width=True)
        
        # Convert the image to RGB
        image_rgb = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)

        # Use the provided function to process the image
        image_with_bounding_box, ocr_result = bounding_box_roi(image_rgb)

        # Display the processed image with bounding boxes
        st.image(image_with_bounding_box, caption="Processed Image with Bounding Boxes", use_column_width=True)
        
        # Display the OCR result
        st.write(f"OCR Result: {ocr_result}")

        # Placeholder: Process the symbol ROI using the pre-trained model
        
        # Assuming symbol ROIs are extracted in your function, and you pass them through the model:
        symbol_roi = extract_symbol_roi(image_rgb)  # Placeholder: Replace this with actual symbol ROI extraction

        # Preprocess the symbol ROI before passing it into the model
        symbol_roi_preprocessed = preprocess_symbol_roi(symbol_roi)  # Example: Resize, normalize, etc.
    

        # Predict the symbol using the model
        symbol_prediction = model.predict(symbol_roi_preprocessed)

        # Interpret the prediction
        predicted_symbol_class = interpret_symbol_prediction(symbol_prediction)  # Example: map prediction to class

        st.write(f"Predicted Symbol Class: {predicted_symbol_class}")

else:
    st.warning("Please enter your name to continue.")